1. Prepare the [Sentenance Transformers](https://github.com/UKPLab/sentence-transformers) package

In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 1.1MB 7.1MB/s 
     |████████████████████████████████| 1.1MB 16.9MB/s 
     |████████████████████████████████| 3.0MB 25.2MB/s 
     |████████████████████████████████| 890kB 42.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.7.2-cp36-none-any.whl size=91190 sha256=7c8d91f54ac1500b9dfff3d5966ad75ebb5e5331b48a3f996f0898d8131921ef
  Stored in directory: /root/.cache/pip/wheels/4b/8f/23/7f93e91176acc7c2d651b54fa9f01e3624a47904145c4d133e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=cd7549a535f63e454eef26f1ada2847c89075c08b630dabd560fbb666db4749a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


Compare encoded vector with cosine distance

In [ ]:
from scipy.spatial import distance
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
print(distance.cosine(transformer.encode('I wanna go home'), transformer.encode('I am going home')))
print(distance.cosine(transformer.encode('I wanna go home'), transformer.encode('I am leaving')))


100%|██████████| 245M/245M [00:11<00:00, 21.8MB/s]


0.11797666549682617
0.5040630102157593


Vector arithmetics

In [ ]:
distance.cosine(transformer.encode('Queen'), transformer.encode('King') - transformer.encode('Male') + transformer.encode('Female'))

0.24157464504241943

Mount Google Drive to save the result

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!mkdir -p "/content/gdrive/My Drive/Colab Data/ch5"

In [ ]:
!ln -s "/content/gdrive/My Drive/Colab Data/ch5" "/content/data"

Read the YouTube comments

In [ ]:
import pandas as pd

df = pd.read_csv('data/data.csv', encoding="utf-8")
df

,text
0,Best to you & Mrs. Will be praying. We love yo...
1,"Godspeed, Mr. President!"
2,Praying for a speedy recovery for you and the ...
3,Trump is bad
4,So glad to see you doing well!! Thank you for...
...,...
8938,You have our prayers and support
8939,How many comments have you deleted?
8940,"on this channel, none! The us is full of Freed..."
8941,"faze joe yeah cool, but I haven’t seen any com..."


Encode the comments into vectors

In [ ]:
import numpy as np
import os

filename="data/bottleneck.dat"
if os.path.exists(filename):
    fp = np.memmap(filename, dtype='float32', mode='r', shape=(8943,768))
else:
    from sentence_transformers import SentenceTransformer
    transformer = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    transformer.encode(df['text'])
    encodings = transformer.encode(df['text'])
    fp = np.memmap(filename, dtype='float32', mode='w+', shape=encodings.shape)
    fp[:] = encodings[:]
    print(encodings.shape)


In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=0.3, min_samples=3, metric='cosine')
result = clustering.fit_predict(fp)
max(result)

32

In [ ]:
df['result'] = result
df.to_csv('data/clustered_result.csv', encoding="utf-8")
df['result'].value_counts()

 0     6061
-1     2711
 1       28
 4       15
 3       11
 17       8
 15       8
 6        7
 12       6
 5        6
 8        5
 18       5
 23       4
 13       4
 9        4
 22       4
 16       4
 10       4
 32       3
 24       3
 25       3
 2        3
 31       3
 26       3
 11       3
 19       3
 20       3
 28       3
 21       3
 29       3
 14       3
 30       3
 7        3
 27       3
Name: result, dtype: int64

Save the vectors and metadata for projector

In [ ]:
vectors = pd.DataFrame(fp)
vectors.to_csv('data/vectors.tsv', sep='\t', header=False, index=False)

meta = df.copy()
meta['text'] = meta['text'].apply(lambda x: x.split('\n')[0])
meta.to_csv('data/meta.tsv', sep='\t', index=False)


Clustering with specific number of clusters

In [ ]:
distance.euclidean(transformer.encode('Queen'), transformer.encode('King') - transformer.encode('Male') + transformer.encode('Female'))

11.848694801330566

In [ ]:
from sklearn.cluster import Birch
clustering = Birch(threshold=12.7, n_clusters=None)
result = clustering.fit_predict(fp)
df['result'] = result
df['result'].value_counts()

0      981
1      916
7      640
193    598
2      577
      ... 
290      1
189      1
274      1
197      1
171      1
Name: result, Length: 291, dtype: int64

In [ ]:
vc = df['result'].value_counts()
result[df['result'].isin(vc[vc <= 10].index.values)]=-1
df['result'] = result

In [ ]:
df.to_csv('data/clustered_result.csv', encoding="utf-8")

In [ ]:
len(df['result'].value_counts())

43

In [ ]:
df['result'].value_counts()

 0      981
 1      916
 7      640
 193    598
 2      577
-1      500
 11     479
 222    469
 256    405
 5      337
 18     335
 138    334
 261    299
 247    253
 199    238
 262    174
 6      139
 279    134
 282    130
 40     127
 36      94
 67      93
 12      86
 95      81
 174     76
 275     66
 20      39
 146     33
 123     31
 96      29
 48      29
 140     25
 286     24
 141     23
 202     22
 253     20
 270     18
 28      18
 108     18
 152     16
 213     13
 209     13
 251     11
Name: result, dtype: int64

In [ ]:
meta = df.copy()
meta['text'] = meta['text'].apply(lambda x: x.split('\n')[0])
meta.to_csv('data/meta.tsv', sep='\t', index=False)

In [ ]:
clustering = Birch(threshold=10, n_clusters=15)
result = clustering.fit_predict(fp)
df['result'] = result
df['result'].value_counts()
df.to_csv('data/clustered_result.csv', encoding="utf-8")
meta = df.copy()
meta['text'] = meta['text'].apply(lambda x: x.split('\n')[0])
meta.to_csv('data/meta.tsv', sep='\t', index=False)

In [ ]:
df['result'].value_counts()

13    2032
1     1565
8     1365
6      594
9      506
12     454
0      450
14     418
4      375
5      308
2      230
3      212
7      171
10     142
11     121
Name: result, dtype: int64